## EDA on NYC Taxi data

In [1]:
# Import required packages
import boto3
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, DoubleType
import pyspark.sql.functions as F

In [2]:
# Create a local spark session
spark = SparkSession.builder \
        .appName('nyc-taxi-data') \
        .getOrCreate()

In [3]:
# Set parameters 
bucket_name = "nyc-tlc" # s3 bucket name with required nyc tlc files

In [4]:
# Create function to read S3 bucket
def list_bucket_contents(bucket, match=''):
    files = []
    s3_resource = boto3.resource('s3')
    bucket_resource = s3_resource.Bucket(bucket)
    for key in bucket_resource.objects.all():
        if match in key.key:
            files.append(key.key)
    return files

In [5]:
# Read yellow and green taxi data for respective years
colours = ["yellow","green"]
years = ["2019","2020"]
files = []

for year in years:
    for colour in colours:
        match = colour + "_tripdata_" + year
        files.extend(list_bucket_contents(bucket=bucket_name, match=match))

In [6]:
files

['trip data/yellow_tripdata_2019-01.csv',
 'trip data/yellow_tripdata_2019-02.csv',
 'trip data/yellow_tripdata_2019-03.csv',
 'trip data/yellow_tripdata_2019-04.csv',
 'trip data/yellow_tripdata_2019-05.csv',
 'trip data/yellow_tripdata_2019-06.csv',
 'trip data/yellow_tripdata_2019-07.csv',
 'trip data/yellow_tripdata_2019-08.csv',
 'trip data/yellow_tripdata_2019-09.csv',
 'trip data/yellow_tripdata_2019-10.csv',
 'trip data/yellow_tripdata_2019-11.csv',
 'trip data/yellow_tripdata_2019-12.csv',
 'trip data/green_tripdata_2019-01.csv',
 'trip data/green_tripdata_2019-02.csv',
 'trip data/green_tripdata_2019-03.csv',
 'trip data/green_tripdata_2019-04.csv',
 'trip data/green_tripdata_2019-05.csv',
 'trip data/green_tripdata_2019-06.csv',
 'trip data/green_tripdata_2019-07.csv',
 'trip data/green_tripdata_2019-08.csv',
 'trip data/green_tripdata_2019-09.csv',
 'trip data/green_tripdata_2019-10.csv',
 'trip data/green_tripdata_2019-11.csv',
 'trip data/green_tripdata_2019-12.csv',
 'tr

In [7]:
# Read January 2019 yellow taxi cab data from S3 bucket
yellow_df = spark.read.csv(f"s3a://{bucket_name}/trip data/yellow_tripdata_2020-07.csv", header=True)

In [8]:
# Show first twenty rows of the imported file
yellow_df.show(20)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2020-07-01 00:25:32|  2020-07-01 00:33:39|              1|         1.50|         1|                 N|         238|          75|           2|          8|  0.5|    0.5|         0|           0|                  0.3

In [9]:
# Print schema of data frame to show field data types and nullability
yellow_df.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)



In [10]:
# Read January 2019 green taxi cab data from S3 bucket
green_df = spark.read.csv(f"s3a://{bucket_name}/trip data/green_tripdata_2019-01.csv", header=True)

In [11]:
green_df.show(10)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2018-12-21 15:17:29|  2018-12-21 15:18:57|                 N|         1|         264|         264|              5|          .00|          3|  0.5|    0.

In [12]:
# Print schema of data frame to show field data types and nullability
green_df.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- trip_type: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)



## Recommended data type changes

All fields imported as a string. The following data type conversions are required:

* VendorID: string -> categorical (drop - needs processing intensive one hot encoding)
* lpep_pickup_datetime: string -> timestamp
* lpep_dropoff_datetime: string -> timestamp
* store_and_fwd_flag: string -> integer (drop)
* RatecodeID: string -> integer
* PULocationID: string -> categorical (drop - needs processing intensive one hot encoding)
* DOLocationID: string -> categorical (drop - needs processing intensive one hot encoding)
* passenger_count: string -> integer
* trip_distance: string -> double
* fare_amount: string -> double
* extra: string -> double
* mta_tax: string -> double 
* tip_amount: string -> double
* tolls_amount: string -> double
* ehail_fee: string -> double 
* improvement_surcharge: string -> double 
* total_amount: string -> double
* payment_type: string -> integer
* trip_type: string -> integer (drop - is not in other data set)
* congestion_surcharge: string -> double (drop - is not in other data set and should be included in extras per dictionary)

In [13]:
# Determine if there are any drop offs before pickups
yellow_df.withColumn("pickup_datetime", F.unix_timestamp(F.col("tpep_pickup_datetime"), "yyyy-MM-dd' 'HH:mm:ss").cast("timestamp")).\
    withColumn("dropoff_datetime", F.unix_timestamp(F.col("tpep_dropoff_datetime"), "yyyy-MM-dd' 'HH:mm:ss").cast("timestamp")).\
    withColumn("trip_duration", (F.col("dropoff_datetime").cast("long") - F.col("pickup_datetime").cast("long"))).\
    filter(F.col("trip_duration") < 0).\
    select(["pickup_datetime","dropoff_datetime","trip_duration"]).\
    show()

+-------------------+-------------------+-------------+
|    pickup_datetime|   dropoff_datetime|trip_duration|
+-------------------+-------------------+-------------+
|2020-07-13 03:14:35|2020-07-01 08:44:59|     -1016976|
|2020-07-18 20:11:06|2019-07-18 20:47:07|    -31620239|
|2020-07-29 17:07:55|2020-07-29 17:07:30|          -25|
|2020-07-27 17:07:50|2020-07-27 17:07:35|          -15|
|2020-07-30 18:07:57|2020-07-30 18:07:12|          -45|
|2020-07-30 00:07:54|2020-07-30 00:07:03|          -51|
+-------------------+-------------------+-------------+



In [14]:
yellow_df.withColumn("pickup_datetime", F.unix_timestamp(F.col("tpep_pickup_datetime"), "yyyy-MM-dd' 'HH:mm:ss").cast("timestamp")).\
    withColumn("dropoff_datetime", F.unix_timestamp(F.col("tpep_dropoff_datetime"), "yyyy-MM-dd' 'HH:mm:ss").cast("timestamp")).\
    withColumn("trip_duration", (F.col("dropoff_datetime").cast("long") - F.col("pickup_datetime").cast("long"))).\
    filter(F.col("trip_duration") < 30).\
    select(["tpep_pickup_datetime","pickup_datetime","tpep_dropoff_datetime","dropoff_datetime","trip_duration","fare_amount"]).\
    show()

+--------------------+-------------------+---------------------+-------------------+-------------+-----------+
|tpep_pickup_datetime|    pickup_datetime|tpep_dropoff_datetime|   dropoff_datetime|trip_duration|fare_amount|
+--------------------+-------------------+---------------------+-------------------+-------------+-----------+
| 2020-07-01 00:03:34|2020-07-01 00:03:34|  2020-07-01 00:03:42|2020-07-01 00:03:42|            8|         52|
| 2020-07-01 00:16:31|2020-07-01 00:16:31|  2020-07-01 00:16:41|2020-07-01 00:16:41|           10|        2.5|
| 2020-07-01 00:16:31|2020-07-01 00:16:31|  2020-07-01 00:16:43|2020-07-01 00:16:43|           12|        2.5|
| 2020-07-01 00:50:08|2020-07-01 00:50:08|  2020-07-01 00:50:18|2020-07-01 00:50:18|           10|         52|
| 2020-07-01 00:09:00|2020-07-01 00:09:00|  2020-07-01 00:09:22|2020-07-01 00:09:22|           22|        2.5|
| 2020-07-01 00:51:31|2020-07-01 00:51:31|  2020-07-01 00:51:39|2020-07-01 00:51:39|            8|        2.5|
|

In [15]:
# Look at payment types 6 = voided trips
yellow_df.filter(F.col("payment_type").astype(IntegerType()) == 6).\
    count()

0

In [11]:
# Look at payment types 4 = dispute
yellow_df.filter(F.col("payment_type").astype(IntegerType()) == 4).\
    count()

3312

In [20]:
# Look at payment types misclassified above 6
yellow_df.filter(F.col("payment_type").astype(IntegerType()) > 6).\
    count()

0

In [21]:
# Look at RatecodeID misclassified above 6
yellow_df.filter(F.col("RatecodeID").astype(IntegerType()) > 6).\
    count()

104

In [12]:
# Look at mta tax consistently $0.50
yellow_df.filter(F.col("mta_tax").astype(DoubleType()) != 0.50).\
    count()


8988

In [13]:
# Look for less than one passenger
yellow_df.filter(F.col("passenger_count").astype(IntegerType()) < 1).\
    count()

19506

In [14]:
# Look for tip amount less than zero
yellow_df.filter(F.col("tip_amount").astype(DoubleType()) < 0).\
    count()

85

In [15]:
# Look for tolls_amount less than zero
yellow_df.filter(F.col("tolls_amount").astype(DoubleType()) < 0).\
    count()

73

In [24]:
# Look for improvement_surcharge consistently $0.30
yellow_df.filter(F.col("improvement_surcharge").astype(DoubleType()) != 0.30).\
    count()

4470

In [17]:
# Look for fare_amount less than zero
yellow_df.filter(F.col("fare_amount").astype(DoubleType()) < 0).\
    count()

3710

In [27]:
# Look for extra greater than $1 (congestion or night time fee)
yellow_df.filter(F.col("extra").astype(DoubleType()) > 3.5).\
    count()


1416

In [19]:
# Look for trip_distance less than zero
yellow_df.filter(F.col("trip_distance").astype(DoubleType()) < 0).\
    count()


0

In [28]:
# Look for trip_distance less than zero
yellow_df.filter(F.col("ehail_fee").astype(DoubleType()) >= 0).\
    count()

AnalysisException: "cannot resolve '`ehail_fee`' given input columns: [passenger_count, tolls_amount, VendorID, congestion_surcharge, PULocationID, DOLocationID, total_amount, tpep_dropoff_datetime, payment_type, tip_amount, tpep_pickup_datetime, extra, mta_tax, store_and_fwd_flag, fare_amount, RatecodeID, trip_distance, improvement_surcharge];;\n'Filter (cast('ehail_fee as double) >= 0)\n+- Relation[VendorID#10,tpep_pickup_datetime#11,tpep_dropoff_datetime#12,passenger_count#13,trip_distance#14,RatecodeID#15,store_and_fwd_flag#16,PULocationID#17,DOLocationID#18,payment_type#19,fare_amount#20,extra#21,mta_tax#22,tip_amount#23,tolls_amount#24,improvement_surcharge#25,total_amount#26,congestion_surcharge#27] csv\n"